In [92]:
import pandas as pd
import json

dataframe = pd.read_json("F:/Spring 18/SPM/HW5/SPM587SP18issues.json", lines=True)

s = dataframe.to_json(orient='records')

cont = json.loads(s)



In [93]:
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])


In [94]:
actions = []
for data in cont:
    action = {
        "_index": "spm_issue",
        "_type": "issues",
        "_id": data['issue_number'],
        "_source": data
    }
    actions.append(action)
    
helpers.bulk(es, actions)

(255, [])

In [95]:
doc = {
    'size' : 255,
    'query' : {
        'match_all' : {}
    }
}
result_without_condition = es.search(index='spm_issue', body=doc,scroll='1h')

In [96]:
sid = result_without_condition['_scroll_id']
scroll_size = result_without_condition['hits']['total']

In [97]:
count= 0
final_location=[]
while(scroll_size>0):
    for document in result_without_condition['hits']['hits']:
        location=[]
        results=document['_source']
        labels = results['labels']
        
        for a in results['labels']:
            Tlabel = a.split(':')[0]
            if(Tlabel=='Latitude' or Tlabel=='Longitude'):
                
                location.append(float(a.split(':')[1]))
                
                if(len(location)>1):
                    final_location.append(location)
                
                
        count= count+1
        
    result_without_condition= es.scroll(scroll_id=sid, scroll ='2m')
    sid = result_without_condition['_scroll_id']
    scroll_size= len(result_without_condition['hits']['hits'])
    print("total count",count)
    #print(final_location)                        
            


('total count', 255)


In [98]:
import folium
from folium.plugins import HeatMap

OP=folium.Map([41.917164,-87.686965],zoom_start=11)
OP

In [99]:
OP.add_child(HeatMap(final_location))


In [ ]:
#output

In [ ]:
#HeatMap(final_location_11).add_to(without_condition_heatmap)

In [ ]:
#without_condition_heatmap

In [100]:
doc1 = {
        'size' : 255,
        'query': {
            'bool': {
                    'must' : [
                        
                              {"query_string": {
                                            "query": "DetectionPhase*Field AND Priority*Critical", 
                                            "fields": ["labels"] 
                                             }
                            }
                              
           
                            ]
                   }
                }
        }
conditional_result_one=es.search(index='issues',body=doc1, scroll = "1h")
conditional_result_one

{u'_scroll_id': u'DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAH2FmJkZ29yLXgwUkMtLXVmejhCNjF5MkEAAAAAAAAB9RZiZGdvci14MFJDLS11Zno4QjYxeTJBAAAAAAAAAfcWYmRnb3IteDBSQy0tdWZ6OEI2MXkyQQAAAAAAAAH4FmJkZ29yLXgwUkMtLXVmejhCNjF5MkEAAAAAAAAB-RZiZGdvci14MFJDLS11Zno4QjYxeTJB',
 u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'52',
    u'_index': u'issues',
    u'_score': 2.0,
    u'_source': {u'Author': u'HSP18SCM69D',
     u'State': u'open',
     u'closed_at': None,
     u'created_at': u'2018-04-09',
     u'issue_number': 52,
     u'labels': [u'Address:645 N MCCLURG CT',
      u'Category:Inquiry',
      u'DetectionPhase:Field',
      u'Latitude:41.893230',
      u'Longitude:-87.617419',
      u'OriginationPhase:Field',
      u'Priority:Critical',
      u'Status:inProgress']},
    u'_type': u'issue'},
   {u'_id': u'26',
    u'_index': u'issues',
    u'_score': 2.0,
    u'_source': {u'Author': u'HSP18SCM69D',
     u'State': u'open',
     u'closed_at': None,
 

In [101]:
sid = conditional_result_one['_scroll_id']
scroll_size = conditional_result_one['hits']['total']

In [102]:
count= 0
final_location_1=[]
while(scroll_size>0):
    for document in conditional_result_one['hits']['hits']:
        location=[]
        results=document['_source']
        labels = results['labels']
        
        for a in results['labels']:
            Tlabel = a.split(':')[0]
            if(Tlabel=='Latitude' or Tlabel=='Longitude'):
                
                location.append(float(a.split(':')[1]))
                
                if(len(location)>1):
                    final_location_1.append(location)
                
                
        count= count+1
        
    conditional_result_one= es.scroll(scroll_id=sid, scroll ='2m')
    sid = conditional_result_one['_scroll_id']
    scroll_size= len(conditional_result_one['hits']['hits'])
    print("total count",count) 

('total count', 4)


In [103]:
OP1=folium.Map([41.917164,-87.686965],zoom_start=11)
OP1.add_child(HeatMap(final_location_1))


In [104]:
doc2 = {
        'size' : 255,
        'query': {
            'bool': {
                    'must' : [
                        
                              {"query_string": {
                                            "query": "DetectionPhase*Field AND Status*Completed", 
                                            "fields": ["labels"] 
                                             }
                            }
                              
           
                            ]
                   }
                }
        }
conditional_result_two=es.search(index='issues',body=doc2, scroll = "1h")
conditional_result_two

{u'_scroll_id': u'DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAH7FmJkZ29yLXgwUkMtLXVmejhCNjF5MkEAAAAAAAAB-hZiZGdvci14MFJDLS11Zno4QjYxeTJBAAAAAAAAAfwWYmRnb3IteDBSQy0tdWZ6OEI2MXkyQQAAAAAAAAH9FmJkZ29yLXgwUkMtLXVmejhCNjF5MkEAAAAAAAAB_hZiZGdvci14MFJDLS11Zno4QjYxeTJB',
 u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'22',
    u'_index': u'issues',
    u'_score': 2.0,
    u'_source': {u'Author': u'HSP18SCM69D',
     u'State': u'open',
     u'closed_at': None,
     u'created_at': u'2018-04-09',
     u'issue_number': 22,
     u'labels': [u'Address:233 W JACKSON',
      u'Category:Bug',
      u'DetectionPhase:Field',
      u'Latitude:40.170101',
      u'Longitude:-92.177847',
      u'OriginationPhase:Coding',
      u'Priority:High',
      u'Status:Completed']},
    u'_type': u'issue'},
   {u'_id': u'42',
    u'_index': u'issues',
    u'_score': 2.0,
    u'_source': {u'Author': u'HSP18SCM69D',
     u'State': u'open',
     u'closed_at': None,
     u'creat

In [105]:
sid = conditional_result_two['_scroll_id']
scroll_size = conditional_result_two['hits']['total']

In [106]:
count= 0
final_location_2=[]
while(scroll_size>0):
    for document in conditional_result_two['hits']['hits']:
        location=[]
        results=document['_source']
        labels = results['labels']
        
        for a in results['labels']:
            Tlabel = a.split(':')[0]
            if(Tlabel=='Latitude' or Tlabel=='Longitude'):
                
                location.append(float(a.split(':')[1]))
                
                if(len(location)>1):
                    final_location_2.append(location)
                
                
        count= count+1
        
    conditional_result_two= es.scroll(scroll_id=sid, scroll ='2m')
    sid = conditional_result_two['_scroll_id']
    scroll_size= len(conditional_result_two['hits']['hits'])
    print("total count",count) 

('total count', 4)


In [107]:
OP2=folium.Map([41.917164,-87.686965],zoom_start=11)
OP2.add_child(HeatMap(final_location_2))

In [108]:
doc3 = {
        'size' : 255,
        'query': {
            'bool': {
                    'must' : [
                        
                              {"query_string": {
                                            "query": "DetectionPhase*Field AND Priority*Critical AND Status*Approved",
                                            "fields": ["labels"]
                                             }
                            }
                              
           
                            ]
                   }
                }
        }
conditional_result_three=es.search(index='issues',body=doc3, scroll = "1h")
conditional_result_three

{u'_scroll_id': u'DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAH_FmJkZ29yLXgwUkMtLXVmejhCNjF5MkEAAAAAAAACABZiZGdvci14MFJDLS11Zno4QjYxeTJBAAAAAAAAAgEWYmRnb3IteDBSQy0tdWZ6OEI2MXkyQQAAAAAAAAICFmJkZ29yLXgwUkMtLXVmejhCNjF5MkEAAAAAAAACAxZiZGdvci14MFJDLS11Zno4QjYxeTJB',
 u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'26',
    u'_index': u'issues',
    u'_score': 3.0,
    u'_source': {u'Author': u'HSP18SCM69D',
     u'State': u'open',
     u'closed_at': None,
     u'created_at': u'2018-04-09',
     u'issue_number': 26,
     u'labels': [u'Address:1951 N WESTERN AVE',
      u'Category:Enhancement',
      u'DetectionPhase:Field',
      u'Latitude:41.917164',
      u'Longitude:-87.686965',
      u'OriginationPhase:Requirements',
      u'Priority:Critical',
      u'Status:Approved']},
    u'_type': u'issue'},
   {u'_id': u'12',
    u'_index': u'issues',
    u'_score': 3.0,
    u'_source': {u'Author': u'HSP18SCM69D',
     u'State': u'open',
     u'closed_a

In [109]:
sid = conditional_result_three['_scroll_id']
scroll_size = conditional_result_three['hits']['total']

In [110]:
count= 0
final_location_3=[]
while(scroll_size>0):
    for document in conditional_result_three['hits']['hits']:
        location=[]
        results=document['_source']
        labels = results['labels']
        
        for a in results['labels']:
            Tlabel = a.split(':')[0]
            if(Tlabel=='Latitude' or Tlabel=='Longitude'):
                
                location.append(float(a.split(':')[1]))
                
                if(len(location)>1):
                    final_location_3.append(location)
                
                
        count= count+1
        
    conditional_result_three= es.scroll(scroll_id=sid, scroll ='2m')
    sid = conditional_result_three['_scroll_id']
    scroll_size= len(conditional_result_three['hits']['hits'])
    print("total count",count) 

('total count', 3)


In [111]:
OP3=folium.Map([41.917164,-87.686965],zoom_start=11)
OP3.add_child(HeatMap(final_location_3))

In [112]:
doc4={
    'size' : 255,
    'query':{
        'bool':{
            'must' :[{'match':{"labels":"DetectionPhase:Field"}},
                     
                     {'bool':{'should':[{'match':{"labels":"Priority:Critical"}},
                                        {'match':{"labels":"Priority:High"}}
                                       ]
                             }
                     },
                     {'bool':{'should':[{'match':{"labels":"Status:Approved"}},
                                        {'match':{"labels":"Status:inProgress"}}
                                       ]
                             }
                     }
                    ]
        }
    }
}
conditional_result_four = es.search(index='issues', body=doc4,scroll='1h')
conditional_result_four

{u'_scroll_id': u'DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAIEFmJkZ29yLXgwUkMtLXVmejhCNjF5MkEAAAAAAAACBRZiZGdvci14MFJDLS11Zno4QjYxeTJBAAAAAAAAAgYWYmRnb3IteDBSQy0tdWZ6OEI2MXkyQQAAAAAAAAIHFmJkZ29yLXgwUkMtLXVmejhCNjF5MkEAAAAAAAACCBZiZGdvci14MFJDLS11Zno4QjYxeTJB',
 u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'396',
    u'_index': u'issues',
    u'_score': 6.9651794,
    u'_source': {u'Author': u'ZSP18SCM00M',
     u'State': u'closed',
     u'closed_at': u'2018-04-13',
     u'created_at': u'2018-04-13',
     u'issue_number': 396,
     u'labels': [u'Category:Inquiry',
      u'DetectionPhase:Field',
      u'OriginationPhase:Coding',
      u'Priority:High',
      u'Status:inProgress']},
    u'_type': u'issue'},
   {u'_id': u'278',
    u'_index': u'issues',
    u'_score': 6.9651794,
    u'_source': {u'Author': u'SSP18SCM86K',
     u'State': u'closed',
     u'closed_at': u'2018-04-13',
     u'created_at': u'2018-04-12',
     u'issue_number': 278,


In [113]:
sid = conditional_result_four['_scroll_id']
scroll_size = conditional_result_four['hits']['total']

In [114]:
count= 0
final_location_4=[]
while(scroll_size>0):
    for document in conditional_result_four['hits']['hits']:
        location=[]
        results=document['_source']
        labels = results['labels']
        
        for a in results['labels']:
            Tlabel = a.split(':')[0]
            if(Tlabel=='Latitude' or Tlabel=='Longitude'):
                
                location.append(float(a.split(':')[1]))
                
                if(len(location)>1):
                    final_location_4.append(location)
                
                
        count= count+1
        
    conditional_result_four= es.scroll(scroll_id=sid, scroll ='2m')
    sid = conditional_result_four['_scroll_id']
    scroll_size= len(conditional_result_four['hits']['hits'])
    print("total count",count) 

('total count', 24)


In [115]:
OP4=folium.Map([41.917164,-87.686965],zoom_start=11)
OP4.add_child(HeatMap(final_location_4))

In [116]:
doc5 = {
        'size' : 255,
        'query': {
            'bool': {
                    'must' : [{'match':{"labels":"Latitude:41.852623"}},
                              {'match':{"labels":"Latitude:41.878693"}},
                              {'match':{"labels":"Latitude:41.893020"}},
                              {'match':{"labels":"Latitude:41.877817"}},
                              {'match':{"labels":"Latitude:41.847095"}},
                              {'match':{"labels":"Latitude:41.838897"}},
                              {'match':{"labels":"Latitude:41.877846"}}
                            ]
                   }
                }
        }
conditional_result_five=es.search(index='issues',body=doc5, scroll = "1h")
conditional_result_five

{u'_scroll_id': u'DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAILFmJkZ29yLXgwUkMtLXVmejhCNjF5MkEAAAAAAAACChZiZGdvci14MFJDLS11Zno4QjYxeTJBAAAAAAAAAgkWYmRnb3IteDBSQy0tdWZ6OEI2MXkyQQAAAAAAAAIMFmJkZ29yLXgwUkMtLXVmejhCNjF5MkEAAAAAAAACDRZiZGdvci14MFJDLS11Zno4QjYxeTJB',
 u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'293',
    u'_index': u'issues',
    u'_score': 8.150893,
    u'_source': {u'Author': u'YSP18SCM40K',
     u'State': u'open',
     u'closed_at': None,
     u'created_at': u'2018-04-12',
     u'issue_number': 293,
     u'labels': [u'Category:Bug',
      u'DetectionPhase:Design',
      u'Latitude:41.852623',
      u'Longitude:-87.611958',
      u'OriginationPhase:Design',
      u'OriginationPhase:Requirements',
      u'Priority:Medium',
      u'Status:Approved',
      u'Status:inProgress',
      u'Status:pendingReview']},
    u'_type': u'issue'},
   {u'_id': u'57',
    u'_index': u'issues',
    u'_score': 7.850109,
    u'_source': {u'Autho

In [117]:
sid = conditional_result_five['_scroll_id']
scroll_size = conditional_result_five['hits']['total']

In [118]:
count= 0
final_location_5=[]
while(scroll_size>0):
    for document in conditional_result_five['hits']['hits']:
        location=[]
        results=document['_source']
        labels = results['labels']
        
        for a in results['labels']:
            Tlabel = a.split(':')[0]
            if(Tlabel=='Latitude' or Tlabel=='Longitude'):
                
                location.append(float(a.split(':')[1]))
                
                if(len(location)>1):
                    final_location_5.append(location)
                
                
        count= count+1
        
    conditional_result_five= es.scroll(scroll_id=sid, scroll ='2m')
    sid = conditional_result_five['_scroll_id']
    scroll_size= len(conditional_result_five['hits']['hits'])
    print("total count",count) 

('total count', 144)


In [119]:
OP5=folium.Map([41.917164,-87.686965],zoom_start=11)
OP5.add_child(HeatMap(final_location_5))